# Introduction

This a short guide on how to generate *Plackett-Burman* designs for screening and computing main effects for a measured response.

## Setup

First, check if you are at the correct project environment. It should be `ExperimentalDesign`:

In [1]:
] st

Project ExperimentalDesign v0.2.0
    Status `~/.julia/dev/ExperimentalDesign/Project.toml`
  [a93c6f00] DataFrames v0.20.2
  [864edb3b] DataStructures v0.17.10
  [31c24e10] Distributions v0.22.6
  [ffbed154] DocStringExtensions v0.8.1
  [e30172f5] Documenter v0.24.6
  [38e38edf] GLM v1.3.7
  [27ebfcd6] Primes v0.4.0
  [2913bbd2] StatsBase v0.32.2
  [3eaba693] StatsModels v0.6.10
  [37e2e46d] LinearAlgebra 
  [56ddb016] Logging 
  [9a3f8284] Random 
  [8dfed614] Test 


Then check if all packages are installed and up to date:

In [2]:
] up

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/dev/ExperimentalDesign/Project.toml`
 [no changes]
  Updating `~/.julia/dev/ExperimentalDesign/Manifest.toml`
 [no changes]


In [3]:
using ExperimentalDesign, GLM, Distributions, DataFrames

# Generating Plackett-Burman Designs

A Plackett-Burman design is an orthogonal design matrix for factors $f_1,\dots,f_N$. Factors are encoded by high and low values, which can be mapped to the interval $[-1, 1]$. For example:

In [4]:
design = PlackettBurman(6)
design.matrix

,factor1,factor2,factor3,factor4,factor5,factor6,dummy1
,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,1,1,1
2,-1,1,-1,1,1,-1,-1
3,1,-1,1,1,-1,-1,-1
4,-1,1,1,-1,-1,-1,1
5,1,1,-1,-1,-1,1,-1
6,1,-1,-1,-1,1,-1,1
7,-1,-1,-1,1,-1,1,1
8,-1,-1,1,-1,1,1,-1


Note that it is not possible to construct exact Plackett-Burman designs for all numbers of factors. In the example above, we needed a seventh extra "dummy" column to construct the design for six factors.

Using the `PlackettBurman` allows quick construction of minimal screening designs for scenarios where we ignore interactions. We can access the underlying formula:

In [5]:
println(design.formula)

response ~ -1 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6 + dummy1


Notice we ignore interactions and include the dummy factor in the model. Strong main effects attributed to dummy factors may indicate important interactions.

We can obtain a tuple with the names of dummy factors:

In [6]:
design.dummy_factors

(:dummy1,)

We can also get the main factors tuple:

In [7]:
design.factors

(:factor1, :factor2, :factor3, :factor4, :factor5, :factor6)

# Computing Main Effects

Suppose that the underlying formula controlling the response variable on the experiments specified in our screening design is given by:

In [8]:
function y(x)
    return (1.2) +
           (2.3 * x[1]) +
           (-3.4 * x[2]) +
           (7.12 * x[3]) +
           (-0.03 * x[4]) +
           (1.1 * x[5]) +
           (-0.5 * x[6]) +
           (1.1 * randn())
end

y (generic function with 1 method)

In [9]:
design.matrix[!, :response] = y.(eachrow(design.matrix[:, collect(design.factors)]))
design.matrix

,factor1,factor2,factor3,factor4,factor5,factor6,dummy1,response
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,1,1,1,1,1,1,1,7.60156
2,-1,1,-1,1,1,-1,-1,-9.05002
3,1,-1,1,1,-1,-1,-1,13.953
4,-1,1,1,-1,-1,-1,1,1.12954
5,1,1,-1,-1,-1,1,-1,-8.2963
6,1,-1,-1,-1,1,-1,1,1.47345
7,-1,-1,-1,1,-1,1,1,-5.40819
8,-1,-1,1,-1,1,1,-1,11.4663


In [10]:
lm(design.formula, design.matrix)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

response ~ 0 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6 + dummy1

Coefficients:
─────────────────────────────────────────────────────────────────────────
          Estimate  Std. Error    t value  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
factor1   2.07425      1.60867   1.28942     0.4199   -18.3659    22.5144
factor2  -3.76248      1.60867  -2.33887     0.2572   -24.2026    16.6776
factor3   6.92894      1.60867   4.30724     0.1452   -13.5112    27.3691
factor4   0.165416     1.60867   0.102828    0.9348   -20.2747    20.6055
factor5   1.26416      1.60867   0.78584     0.5760   -19.176     21.7043
factor6  -0.26782      1.60867  -0.166485    0.8950   -20.7079    20.1723
dummy1   -0.409582     1.60867  -0.254609    0.8413   -20.8497    2

In [11]:
random_design_generator = RandomDesign(tuple(fill(Uniform(-1, 1), 6)...))
random_design = rand(random_design_generator, 8)

,factor1,factor2,factor3,factor4,factor5,factor6
,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.068281,0.27051,0.138395,0.616745,-0.878983,0.613014
2,0.646655,0.876703,-0.0306063,-0.630772,0.238156,0.700154
3,-0.922054,-0.244378,0.323398,-0.215985,-0.213029,0.217699
4,0.940534,0.643173,-0.234738,-0.679713,0.178407,-0.415013
5,-0.887655,0.0764178,0.940784,-0.899612,0.865231,0.481555
6,-0.491294,0.840906,0.53816,-0.566049,0.356879,-0.537592
7,-0.187662,-0.0745744,0.48564,0.48893,-0.540243,-0.0906579
8,0.519675,0.16162,0.571374,-0.131514,0.249054,0.5962


In [12]:
random_design[!, :response] = y.(eachrow(random_design[:, :]))
random_design

,factor1,factor2,factor3,factor4,factor5,factor6,response
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.068281,0.27051,0.138395,0.616745,-0.878983,0.613014,-0.448489
2,0.646655,0.876703,-0.0306063,-0.630772,0.238156,0.700154,-1.94126
3,-0.922054,-0.244378,0.323398,-0.215985,-0.213029,0.217699,0.269011
4,0.940534,0.643173,-0.234738,-0.679713,0.178407,-0.415013,-1.5139
5,-0.887655,0.0764178,0.940784,-0.899612,0.865231,0.481555,6.67367
6,-0.491294,0.840906,0.53816,-0.566049,0.356879,-0.537592,0.875101
7,-0.187662,-0.0745744,0.48564,0.48893,-0.540243,-0.0906579,3.17484
8,0.519675,0.16162,0.571374,-0.131514,0.249054,0.5962,5.10367


In [15]:
lm(random_design_generator.formula, random_design)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

response ~ 1 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6

Coefficients:
───────────────────────────────────────────────────────────────────────────────
               Estimate  Std. Error    t value  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)  -22.2141      34.4849   -0.644167    0.6357   -460.387     415.959
factor1       13.7056      17.9554    0.763314    0.5849   -214.439     241.85 
factor2       -2.53213      1.32228  -1.91498     0.3064    -19.3333     14.269
factor3       47.0808      60.5723    0.777266    0.5794   -722.563     816.725
factor4      -42.4974      67.2713   -0.631732    0.6413   -897.26      812.265
factor5      -48.2423      77.5769   -0.621864    0.6458  -1033.95      937.466
factor6        0.28981

In [14]:
lm(design.formula, design.matrix)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

response ~ 0 + factor1 + factor2 + factor3 + factor4 + factor5 + factor6 + dummy1

Coefficients:
─────────────────────────────────────────────────────────────────────────
          Estimate  Std. Error    t value  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
factor1   2.07425      1.60867   1.28942     0.4199   -18.3659    22.5144
factor2  -3.76248      1.60867  -2.33887     0.2572   -24.2026    16.6776
factor3   6.92894      1.60867   4.30724     0.1452   -13.5112    27.3691
factor4   0.165416     1.60867   0.102828    0.9348   -20.2747    20.6055
factor5   1.26416      1.60867   0.78584     0.5760   -19.176     21.7043
factor6  -0.26782      1.60867  -0.166485    0.8950   -20.7079    20.1723
dummy1   -0.409582     1.60867  -0.254609    0.8413   -20.8497    2